In [1]:
from datasets import load_dataset

dataset = load_dataset("amazon_us_reviews","Books_v1_00")

Reusing dataset amazon_us_reviews (C:\Users\Mahekkumar\.cache\huggingface\datasets\amazon_us_reviews\Books_v1_00\0.1.0\17b2481be59723469538adeb8fd0a68b0ba363bbbdd71090e72c325ee6c7e563)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
import pandas as pd
import numpy as np
from surprise import Dataset
from surprise import Reader
from surprise import NMF
from surprise.model_selection import cross_validate, GridSearchCV

In [3]:
df = pd.DataFrame(dataset['train']['customer_id'])
df['product_id'] = dataset['train']['product_id']
df['star_rating'] = dataset['train']['star_rating']
df

,0,product_id,star_rating
0,25933450,0439873800,5
1,1801372,1623953553,5
2,5782091,142151981X,5
3,32715830,014241543X,5
4,14005703,1604600527,5
...,...,...,...
10319085,52004451,B003ZX1EQS,5
10319086,36099005,0786652918,4
10319087,38081951,1608981215,4
10319088,53040283,0670023418,5


In [4]:
df = df.rename(columns={0:"customer_id"})

In [5]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df[['customer_id','product_id','star_rating']],reader)

In [6]:
%%time
trainset=data.build_full_trainset()

Wall time: 24.6 s


In [7]:
%%time
testset = trainset.build_testset()

Wall time: 9.92 s


In [8]:
param_grid = {'n_factors': [18,20,25]}
gs = GridSearchCV(NMF, param_grid, measures=['RMSE'], cv=5)

In [9]:
gs.fit(data)

In [10]:
model = gs.best_estimator['rmse']

In [15]:
gs.best_params['rmse']

{'n_factors': 25}

In [11]:
%%time
model.fit(trainset)

Wall time: 20min 30s


In [12]:
%%time
pred = model.test(testset)

Wall time: 1min


In [13]:
from surprise import accuracy

In [14]:
accuracy.rmse(pred)

RMSE: 0.2302


0.2301726401763984